In [ ]:
import pandas as pd
import csv

In [ ]:
import ee
ee.Authenticate()
# Initialize the Earth Engine API
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=etl9ijLK5M9g7_t6OBRyOGS6MNbqcuEeploffNXDz0Q&tc=vaBLDhUBCI7Dh99vi6dj6dg-jrrPxHNWvtw-6AxqyTk&cc=aNuvZ6psGJVbxXBtDVqlCFxBPqUCDgs6ZA-QkmfxUAU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXk4rRz91PH8tGd0eeCclXP0Q6_ZQ5v-BNfWhwoeS64mOsYwZw6yDCQ

Successfully saved authorization token.


In [ ]:
# column      row
# [longitude, latitude]

#Hoima [31.351498594, 1.42583163]
#Temp [31.351498594 + 0.03*30, 1.42583163 + 0.08 * 20]
#Temp2 [32.3914985940, -2.8141683700]
#Tanga [39.105554, -5.066]

startPoint = [(-0.05 * 8) + 31.351498594, 1.42583163]     #Hoima
endPoint = [32.403194444, -2.87491666]    #Temp

#startPoint = [32.3914985940, -2.8141683700]  #temp
#endPoint = [39.105554, -5.066 - 15 * 0.08]

In [ ]:
#pointsRaw = []
mapper = { }
#index = 0
graph = { }

def add_node(row, column, startPoint, endPoint, shiftLon, shiftLat):
    max_row = int(abs(startPoint[1] - endPoint[1]) / shiftLat)
    max_collumn = int(abs(startPoint[0] - endPoint[0]) / shiftLon)

    key = str(row) + "_" + str(column)

    if key in graph:
      return

    currentStartPoint = [startPoint[0] + column * shiftLon, startPoint[1] - row *shiftLat]
    arr = []
    if row > 0:
      arr.append(str(row - 1) + "_" + str(column))
    if column > 0:
      arr.append(str(row) + "_" + str(column - 1))
    if row > 0 and column > 0:
      arr.append(str(row - 1) + "_" + str(column - 1))

    if max_collumn >= column:
      arr.append(str(row) + "_" + str(column + 1))
      add_node(row, column + 1, startPoint, endPoint, shiftLon, shiftLat)
      if(row > 0):
        arr.append(str(row - 1) + "_" + str(column + 1))

    if max_row >= row:
      key_right = str(row + 1) + "_" + str(column)
      arr.append(key_right)
      add_node(row + 1, column, startPoint, endPoint, shiftLon, shiftLat)
      if(column > 0):
        arr.append(str(row + 1) + "_" + str(column - 1))

    if max_collumn >= column and max_row >= row:
      key_g = str(row + 1) + "_" + str(column + 1)
      arr.append(key_g)
      add_node(row + 1, column + 1, startPoint, endPoint, shiftLon, shiftLat)

    #pointsRaw.append()
    mapper[key] = currentStartPoint
    #index += 1
    graph[key] = arr

def create_graph(start, end, shiftLon, shiftLat):
    row = 0
    column = 0
    if start[0] > end[0]:
      start, end = end, start
    add_node(0, 0, start, end, shiftLon, shiftLat)

create_graph(startPoint, endPoint, 0.03, 0.08)

graph
#pointsRaw
#mapper

#print("pointsRaw:", len(pointsRaw), "mapper:", len(mapper))

{'54_49': ['53_49', '54_48', '53_48'],
 '53_49': ['52_49', '53_48', '52_48', '54_49', '54_48'],
 '52_49': ['51_49', '52_48', '51_48', '53_49', '53_48'],
 '51_49': ['50_49', '51_48', '50_48', '52_49', '52_48'],
 '50_49': ['49_49', '50_48', '49_48', '51_49', '51_48'],
 '49_49': ['48_49', '49_48', '48_48', '50_49', '50_48'],
 '48_49': ['47_49', '48_48', '47_48', '49_49', '49_48'],
 '47_49': ['46_49', '47_48', '46_48', '48_49', '48_48'],
 '46_49': ['45_49', '46_48', '45_48', '47_49', '47_48'],
 '45_49': ['44_49', '45_48', '44_48', '46_49', '46_48'],
 '44_49': ['43_49', '44_48', '43_48', '45_49', '45_48'],
 '43_49': ['42_49', '43_48', '42_48', '44_49', '44_48'],
 '42_49': ['41_49', '42_48', '41_48', '43_49', '43_48'],
 '41_49': ['40_49', '41_48', '40_48', '42_49', '42_48'],
 '40_49': ['39_49', '40_48', '39_48', '41_49', '41_48'],
 '39_49': ['38_49', '39_48', '38_48', '40_49', '40_48'],
 '38_49': ['37_49', '38_48', '37_48', '39_49', '39_48'],
 '37_49': ['36_49', '37_48', '36_48', '38_49', '3

In [ ]:
from functools import reduce
import geemap

elevation_mask = ee.Image("USGS/SRTMGL1_003")

def get_elevation(coords):
    point = ee.Geometry.Point(coords)
    elevation = elevation_mask.sample(point, 30).first().get('elevation')

    # Check if elevation is available, otherwise set to 0
    return ee.Algorithms.If(elevation, elevation, 0)

# Combine the points and elevations into a feature collection using list comprehension
feature_collection = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point(mapper[key]), {'elevation': get_elevation(mapper[key]), 'key': key}) for key in graph
])

# Export the feature collection to Google Drive as a CSV file
elevation_task = ee.batch.Export.table.toDrive(
    collection=feature_collection,
    description='Elevation',
    folder='EE/Elevation',  # Change to your desired folder on Google Drive
    fileFormat='CSV'
)

# Start the export task
elevation_task.start()

In [ ]:
import time
from pprint import pprint

# Wait for the task to complete
while elevation_task.active():
    # Print task status and information
    time.sleep(10)  # Sleep for 10 seconds before checking again

pprint(elevation_task.status())

In [ ]:
points_water = []
rq = [startPoint[0] - 0.5, startPoint[1] + 0.5, endPoint[0] + 0.5, endPoint[1] - 0.5]
for key in graph:
    rawPoint = mapper[key]
    feature = ee.Feature(ee.Geometry.Point(rawPoint))
    feature = feature.set("key", key)
    points_water.append(feature)

# Load the JRC Global Surface Water dataset
water_occurrence = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
water_occurrence = water_occurrence.clip(ee.Geometry.Rectangle(rq))

# Create a FeatureCollection from the list of rectangles
rectangles_fc = ee.FeatureCollection(points_water)

# Function to calculate average water occurrence for a rectangle
def calculate_average_water_occurrence(feature):
    # Get the rectangle geometry from the feature
    rectangle = ee.Feature(feature).geometry()
    rectangle = rectangle.buffer(100)
    # Clip the image to the current rectangle
    #clipped_image = water_occurrence.clip(rectangle)

    # Calculate average water occurrence in the current rectangle
    avg_water_occurrence = water_occurrence.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=rectangle,
        scale=30  # Adjust the scale according to your dataset
    )

    # Set the result as properties of the feature
    return feature.set(avg_water_occurrence)

# Map the function over the FeatureCollection
result_collection = rectangles_fc.map(calculate_average_water_occurrence)

# Export the result collection to Google Drive as a single CSV file
water_task = ee.batch.Export.table.toDrive(
    collection=result_collection,
    description='AverageWaterOccurrence',
    folder='EE/WaterOccurrence',  # Change this to your desired output folder
    fileFormat='CSV'
)
# Start the batch task
water_task.start()

In [ ]:
import time
from pprint import pprint

# Wait for the task to complete
while water_task.active():
    # Print task status and information
    time.sleep(10)  # Sleep for 10 seconds before checking again

pprint(water_task.status())

{'attempt': 1,
 'batch_eecu_usage_seconds': 43.91474151611328,
 'creation_timestamp_ms': 1698155851930,
 'description': 'AverageWaterOccurrence',
 'destination_uris': ['https://drive.google.com/#folders/1W8hVpsFJu8qwhhPEcrjcRXYklWmTgBDC'],
 'id': 'PSDDBJ75WABZUOEINTDW56MR',
 'name': 'projects/earthengine-legacy/operations/PSDDBJ75WABZUOEINTDW56MR',
 'start_timestamp_ms': 1698155887283,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1698156144029}


In [ ]:
import geemap

polygons = ee.FeatureCollection('WCMC/WDPA/current/polygons')
rect = ee.Geometry.Rectangle([startPoint[0], startPoint[1], endPoint[0], endPoint[1]])
polygons = polygons.filterBounds(rect)

points_temp_shit = []
for key in graph:
    rawPoint = mapper[key]
    feature = ee.Feature(ee.Geometry.Point(rawPoint))
    feature = feature.buffer(10)
    feature = feature.set("key", key)
    points_temp_shit.append(feature)
# Create a set of points as a FeatureCollection
points_protected_temp = ee.FeatureCollection(points_temp_shit)

# Define a function to check if a point is within the polygon
def is_point_in_polygon(feature):
    point = ee.Feature(feature)
    is_inside = ee.Algorithms.If(
        point.geometry().intersects(polygons.geometry(), ee.ErrorMargin(1)),
        True,
        False
    )
    #polygons.geometry().intersection(point.geometry(), 10)

    return point.set('isInside', is_inside)

# Map the function over the points
protected_points = points_protected_temp.map(is_point_in_polygon)

# Export the result to Google Drive
protected_points_task = ee.batch.Export.table.toDrive(
    collection=protected_points,
    description='ProtectedPoints',
    folder='EE/ProtectedPoints',  # Replace with your desired folder in Google Drive
    fileFormat='CSV'
)

protected_points_task.start()

In [ ]:
import time
from pprint import pprint

# Wait for the task to complete
while protected_points_task.active():
    # Print task status and information
    time.sleep(10)  # Sleep for 10 seconds before checking again

pprint(protected_points_task.status())

{'attempt': 1,
 'batch_eecu_usage_seconds': 6.707516193389893,
 'creation_timestamp_ms': 1698126577163,
 'description': 'ProtectedPoints',
 'destination_uris': ['https://drive.google.com/#folders/10c21aMjKgJOfHUByclk1fPqRNIAl389Y'],
 'id': 'YGNOAUVUQJXDDE5MNRSO447Y',
 'name': 'projects/earthengine-legacy/operations/YGNOAUVUQJXDDE5MNRSO447Y',
 'start_timestamp_ms': 1698126589419,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1698126611638}


In [ ]:
elevation_list = {}

elevation_data = pd.read_csv("Elevation.csv")
water_data = pd.read_csv("AverageWaterOccurrence.csv")
protection_data = pd.read_csv("ProtectedPoints.csv")

prev_water_occurrence = 0.0
for key in graph:
    desired_elevation_row = elevation_data.loc[elevation_data["key"] == key]
    desired_water_row = water_data.loc[water_data["key"] == key]
    #print(desired_water_row)
    height = desired_elevation_row["elevation"].values[0]
    water_occurrence_value = desired_water_row["max_extent"].values[0]

    protection_data_row = protection_data.loc[elevation_data["key"] == key]
    is_protected = protection_data_row["isInside"].values[0]

    result = height
    if(is_protected):
        result += 1000000000000
      #print("ad")
    elif (water_occurrence_value >= 1):
        result += 1000000000000
    elevation_list[key] = result
    print("height:", height, "\twater occurrence: {:.4f}".format(water_occurrence_value), "\tisProtected:", is_protected)
    print("key:", key, "\t" ,"elevation:", result, "\n")

Выходные данные были обрезаны до нескольких последних строк (5000).
key: 16_30 	 elevation: 1180 

height: 1188 	water occurrence: 0.0000 	isProtected: False
key: 15_30 	 elevation: 1188 

height: 1162 	water occurrence: 1.0000 	isProtected: False
key: 14_30 	 elevation: 1000000001162 

height: 1167 	water occurrence: 0.0000 	isProtected: False
key: 13_30 	 elevation: 1167 

height: 1208 	water occurrence: 0.0000 	isProtected: False
key: 12_30 	 elevation: 1208 

height: 1261 	water occurrence: 0.0000 	isProtected: False
key: 11_30 	 elevation: 1261 

height: 1242 	water occurrence: 0.0000 	isProtected: False
key: 10_30 	 elevation: 1242 

height: 1152 	water occurrence: 0.0000 	isProtected: False
key: 9_30 	 elevation: 1152 

height: 1203 	water occurrence: 0.0000 	isProtected: False
key: 8_30 	 elevation: 1203 

height: 1136 	water occurrence: 0.0000 	isProtected: False
key: 7_30 	 elevation: 1136 

height: 1091 	water occurrence: 0.0000 	isProtected: False
key: 6_30 	 elevation: 109

In [ ]:
import math

def haversine_distance(coord1, coord2):
    # Radius of the Earth in meters
    R = 6371000.0
    norm_point1 = mapper[coord1]
    norm_point2 = mapper[coord2]
    # Convert decimal degrees to radians
    lat1 = math.radians(norm_point1[1])  # degrees to radians
    lon1 = math.radians(norm_point1[0])
    lat2 = math.radians(norm_point2[1])
    lon2 = math.radians(norm_point2[0])

    # Calculate the differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula for distance
    a = math.sin(dlat / 2.0)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    ## Calculate the horizontal distance
    distance_horizontal = R * c
    #temp = math.sin(lat1) * math.sin(lat2) + math.cos(lat1) * math.cos(lat2) * math.cos(lon2 - lon1)
    #distance_horizontal = math.acos(temp) * R

    return distance_horizontal

def distance_between_points(coord1, coord2):
    # Get the height values from the list
    height1 = elevation_list[coord1]
    height2 = elevation_list[coord2]
    if(height1 >= 1000000000000):
      height1 %= 10000000
    if(height2 >= 1000000000000):
      height2 %= 10000000
    # Calculate the horizontal distance
    horizontal_distance = haversine_distance(coord1, coord2)

    # Calculate the difference in heights
    vertical_distance = abs(height2 - height1)

    # Calculate the total 3D distance
    distance_3d = math.sqrt(horizontal_distance**2 + vertical_distance**2)

    return distance_3d


In [ ]:
import heapq

def dijkstra(start):
    # Initialize distances with infinity for all nodes except the start node
    distances = {node: float('infinity') for node in graph}
    distances[start] = 0

    # Initialize paths with an empty list for all nodes
    paths = {node: [] for node in graph}

    # Priority queue to store (distance, node) pairs
    priority_queue = [(0, start)]

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Check if the current distance is greater than the known distance
        if current_distance > distances[current_node]:
            continue

        # Explore neighbors and update distances and paths
        for neighbor in graph[current_node]:
            height1 = elevation_list[current_node]
            height2 = elevation_list[neighbor]
            weight = 0
            #if (height1 >= 2000000000000 or height2 >= 2000000000000):
            #  weight += 2000000000000000000000
            if (height1 >= 1000000000000 or height2 >= 1000000000000):
              weight += 1000000000000000000000
            else:
              d = 0.592
              g = 9.81
              ro = 870
              u = 1.453
              f = 0.230
              length = distance_between_points(current_node, neighbor)
              change = height2 - height1
              weight += (f * (length / d) * u**2 * ro) / 2 + ro * g * change
            #print("neighbor:", neighbor, "weight:", weight)
            cost = current_distance + weight

            # If a shorter path is found, update the distance and path
            if cost < distances[neighbor]:
                distances[neighbor] = cost
                paths[neighbor] = paths[current_node] + [current_node]
                heapq.heappush(priority_queue, (cost, neighbor))

    return distances, paths
start_key = "0_13"
#start_key = "0_0"

distances, paths = dijkstra(start_key)


In [ ]:
#54_49
end_key = "54_49"
#end_key = "28_224"

govnishe = paths[end_key]
n = len(govnishe)
distances = {}
distances[govnishe[0]] = 0
pressure_losses = {}
pressure_losses[govnishe[0]] = 0
for i in range(0,n):
  if(i + 1 < n):
    dist = distance_between_points(govnishe[i], govnishe[i + 1])
    distances[govnishe[i + 1]] = distances[govnishe[i]] + (dist / 1000)
    d = 0.592
    g = 9.81
    ro = 870
    u = 1.453
    f = 0.230
    height1 = elevation_list[govnishe[i]]
    height2 = elevation_list[govnishe[i + 1]]
    if(height1 >= 1000000000000):
      height1 %= 10000000
    if(height2 >= 1000000000000):
      height2 %= 10000000

    length = dist
    change = height2 - height1
    weight = (f * (length / d) * (u**2) * ro) / 2 + ro * g * change
    pressure_losses[govnishe[i + 1]] =  pressure_losses[govnishe[i]] + weight

In [ ]:
import folium
from ipyleaflet import Map, Marker

my_map = folium.Map(location=[(startPoint[0] + endPoint[0]) / 2, (startPoint[1] + endPoint[1]) / 2], zoom_start=3)

# Add a PolyLine to the map
final_points = []
for path_point in paths[end_key]:
    coords = mapper[path_point]
    final_points.append([coords[1], coords[0]])

folium.PolyLine(locations=final_points, color='blue').add_to(my_map)

f = open("ThirdLine.txt", "w")
for marker in paths[end_key]:
    coords = mapper[marker]
    string = f"key: {marker}\tcoord: {coords[0]:.10f} {coords[1]:.10f}\televation: {elevation_list[marker] % 10000000}\tdistance: {distances[marker]}\tpressure_loss: {pressure_losses[marker]}\n"
    folium.Marker(location=[coords[1], coords[0]], popup=string).add_to(my_map)
    f.write(string)
#    #print("key:", marker, "\tcoord:", f"{coords[0]:.10f} {coords[1]:.10f}", "\televation:", elevation_list[marker])

f.close()
# Display the map
my_map.save("thirdLine.html")  # Save the map as an HTML file
my_map

In [ ]:
#4°45'35.26"Ю  35° 2'47.57"В
print("lat",2 + (45 / 60) + (35.26 / 3600))
print("lon",35 + (2 / 60) + (47.57 / 3600))

lat 2.7597944444444447
lon 35.04654722222222


In [ ]:
import folium
from ipyleaflet import Map, Marker

my_map = folium.Map(location=[(startPoint[1] + endPoint[1]) / 2, (startPoint[0] + endPoint[0]) / 2], zoom_start=9)

for marker in mapper:
    coords = mapper[marker]
    folium.Marker(location=[coords[1], coords[0]], popup=marker).add_to(my_map)
    #print("key:", marker, "coord:", coords)

# Display the map
my_map

In [ ]:
import geemap

# Load the polygon dataset
polygons = ee.FeatureCollection('WCMC/WDPA/current/polygons')

# Visualize the result
Map = geemap.Map()
Map.addLayer(polygons, {'color': '00FF00'}, 'Protected Zone')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…